In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# set number of rows in one plot
r = 8
LABEL = "Q"

In [ ]:
## looking at the signals generated from Classic GANs

from ClassicGAN.modelv2 import Generator
model = Generator(200)

model.load_state_dict(torch.load("ClassicGAN/saved_models/"+LABEL+"/generator.pth", weights_only=True) )
data = pd.read_pickle("data/"+LABEL+".pkl")
X = torch.tensor(data["beat"].to_list())

noise = torch.randn((r**2,200,1))

with torch.no_grad():
    pred = model(noise)
pred = pred.detach().cpu().squeeze(1).numpy()[:].transpose()
    

fig, axs = plt.subplots(r,r)
fig.set_size_inches(15, 10)
fig.suptitle('Classic GAN')
for i in range(r):
    for j in range(r):
        axs[i,j].plot(pred[:,r*i + j], label="Synthetic")
        axs[i,j].plot(X[np.random.choice(len(X))], label = "Real")
        axs[i,j].axis('off')

handles, labels = axs[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center' ,ncols = 2)
# plt.show()
plt.savefig("images/"+LABEL+"/ClassicGAN.png")
plt.close()

In [ ]:
## looking at the signals generated from GANs with Custom Loss
from GAN_CustomLoss.modelv2 import Generator
model = Generator(200)

model.load_state_dict(torch.load("GAN_CustomLoss/saved_models/"+LABEL+"/generator.pth", weights_only=True) )
data = pd.read_pickle("data/"+LABEL+".pkl")
X = torch.tensor(data["beat"].to_list())

noise = torch.randn((r*r,200,1))

with torch.no_grad():
    pred = model(noise)

pred = pred.detach().cpu().squeeze(1).numpy()[:].transpose()

    

fig, axs = plt.subplots(r,r)
fig.set_size_inches(15, 10)
fig.suptitle('GANs with Custom Loss')
for i in range(r):
    for j in range(r):
        axs[i,j].plot(pred[:,r*i + j], label="Synthetic")
        axs[i,j].plot(X[np.random.choice(len(X))], label = "Real")
        axs[i,j].axis('off')

handles, labels = axs[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center' ,ncols = 2)
# plt.show()
plt.savefig("images/"+LABEL+"/GAN_CustomLoss.png")
plt.close()

In [ ]:
## looking at the signals generated from MD-GAN
from MD_GAN.modelv2 import Generator
model = Generator(200)

model.load_state_dict(torch.load("MD_GAN/saved_models/"+LABEL+"/generator.pth", weights_only=True) )
data = pd.read_pickle("data/"+LABEL+".pkl")
X = torch.tensor(data["beat"].to_list())

noise = torch.randn((r**2,200,1))

with torch.no_grad():
    pred = model(noise)
pred = pred.detach().cpu().squeeze(1).numpy()[:].transpose()
    

fig, axs = plt.subplots(r,r)
fig.set_size_inches(15, 10)
fig.suptitle('MD-GAN')
for i in range(r):
    for j in range(r):
        axs[i,j].plot(pred[:,r*i + j], label="Synthetic")
        axs[i,j].plot(X[np.random.choice(len(X))], label = "Real")
        axs[i,j].axis('off')

handles, labels = axs[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center' ,ncols = 2)
# plt.show()
plt.savefig("images/"+LABEL+"/MD-GAN.png")
plt.close()

In [3]:
# comparing metrics of generated samples to upper-bound
from MD_GAN.metrics import *
from MD_GAN.utils import utility
from tslearn.metrics import SoftDTWLossPyTorch



def return_samples(LABEL, data):
    index = {"F": [a for a in range(417,712)],
             "N": [a for a in range(2,1002)],
             "Q": [a for a in range(46,1133)],
             "S": [a for a in range(1953, 2721)],
             "V": [a for a in range(854, 1295)]}
    
    y = data[index[LABEL]]
    y = y.reshape(y.shape[0], 1, y.shape[-1])
    
    return torch.tensor(y)


data = pd.read_pickle("data/"+LABEL+".pkl")
    
tmp = np.array(data["beat"].to_list())
# tmp = sc.fit_transform(tmp.T).T

samples = return_samples(LABEL,tmp)

del tmp




class pipeline:
    def __init__(self, metric_size):
        self.utils = utility()
        self.dtw = SoftDTWLossPyTorch(gamma=0.01)
        self.metric_size = metric_size
        
    def process(self, input, samples):
        # Convert images once, not repeatedly
        input_imgs = self.utils.convert(input.squeeze(1))
        sample_imgs = self.utils.convert(samples.squeeze(1))

        batch_size, sample_size = input_imgs.shape[0], sample_imgs.shape[0]
        
        # Pre-allocate metrics tensor
        metrics = torch.zeros((batch_size, self.metric_size, sample_size), device=input.device)
        
        # Vectorize outer loop
        for s in range(sample_size):
            grayB = sample_imgs[s]  # Shared across batch
            sigB = samples[s].reshape(1, -1, 1)  # Shared across batch
            
            # Parallelize batch computation
            grayA = input_imgs  # All input images
            sigA = input.reshape(batch_size, -1, 1)  # All input signals
            
            # Precompute DTW for the entire batch
            dtw_values = torch.stack([self.dtw(sigA[b].reshape(1,-1,1), sigB) for b in range(batch_size)])
            
            # Compute all metrics for the batch
            metric_results = torch.stack([torch.tensor([UQI.process(grayA[b], grayB),
                                          VIFP.process(grayA[b], grayB),
                                          SCC.process(grayA[b], grayB),
                                          SAM.process(grayA[b], grayB),
                                          ERGAS.process(grayA[b], grayB),
                                          RASE.process(grayA[b], grayB),
                                          SIFT.process(grayA[b], grayB),
                                          SSIM.process(grayA[b], grayB),
                                          dtw_values[b]]) for b in range(batch_size)], dim=0)
            
            metrics[:, :, s] = metric_results

        # Take the mean across the sample dimension
        # metrics = metrics.mean(dim=2)
        return metrics
    
pip = pipeline(9)

c:\Users\adity\anaconda3\envs\cs6140\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [19]:
# random signal
from ClassicGAN.modelv2 import Generator
from collections import defaultdict

model = Generator(200)
# model.load_state_dict(torch.load("ClassicGAN/saved_models/"+LABEL+"/generator.pth", weights_only=True) )

summ = defaultdict(list)

for i in tqdm(range(20)):
    synthetic = model(torch.randn((8,200,1)))
    
    tmp = pip.process(synthetic.detach().cpu(), samples[np.random.choice(samples.shape[0])].reshape(1, 1, 280).detach().cpu())
    
    
    summ["UQI"] += torch.Tensor.tolist(tmp[:,0].reshape(1,-1))[0]
    summ["VIFP"] += torch.Tensor.tolist(tmp[:,1].reshape(1,-1))[0]
    summ["SCC"] += torch.Tensor.tolist(tmp[:,2].reshape(1,-1))[0]
    summ["SAM"] += torch.Tensor.tolist(tmp[:,3].reshape(1,-1))[0]
    summ["ERGAS"] += torch.Tensor.tolist(tmp[:,4].reshape(1,-1))[0]
    summ["RASE"] += torch.Tensor.tolist(tmp[:,5].reshape(1,-1))[0]
    summ["SIFT"] += torch.Tensor.tolist(tmp[:,6].reshape(1,-1))[0]
    summ["SSIM"] += torch.Tensor.tolist(tmp[:,7].reshape(1,-1))[0]
    summ["DTW"] += torch.Tensor.tolist(tmp[:,8].reshape(1,-1))[0]
    
summ = pd.DataFrame(summ)
print("Metrics of Random signals")
print(summ.mean())
print(summ.min())
print(summ.max())

100%|██████████| 20/20 [01:56<00:00,  5.83s/it]

Metrics of Random signals
UQI         0.940493
VIFP        0.001485
SCC         0.000033
SAM         0.229594
ERGAS       6.187481
RASE     1230.765601
SIFT       10.431250
SSIM        0.759256
DTW      1274.810290
dtype: float64
UQI         0.927322
VIFP        0.000127
SCC        -0.002312
SAM         0.209199
ERGAS       5.538191
RASE     1015.576233
SIFT        0.000000
SSIM        0.708632
DTW       843.380066
dtype: float64
UQI         0.950891
VIFP        0.010624
SCC         0.001954
SAM         0.253936
ERGAS       6.966201
RASE     1495.468994
SIFT       30.000000
SSIM        0.796823
DTW      1784.559326
dtype: float64


In [18]:
from ClassicGAN.modelv2 import Generator
from collections import defaultdict

model = Generator(200)
model.load_state_dict(torch.load("ClassicGAN/saved_models/"+LABEL+"/generator.pth", weights_only=True) )

summ = defaultdict(list)

for i in tqdm(range(20)):
    synthetic = model(torch.randn((8,200,1)))
    
    tmp = pip.process(synthetic.detach().cpu(), samples[np.random.choice(samples.shape[0])].reshape(1, 1, 280).detach().cpu())
    
    
    summ["UQI"] += torch.Tensor.tolist(tmp[:,0].reshape(1,-1))[0]
    summ["VIFP"] += torch.Tensor.tolist(tmp[:,1].reshape(1,-1))[0]
    summ["SCC"] += torch.Tensor.tolist(tmp[:,2].reshape(1,-1))[0]
    summ["SAM"] += torch.Tensor.tolist(tmp[:,3].reshape(1,-1))[0]
    summ["ERGAS"] += torch.Tensor.tolist(tmp[:,4].reshape(1,-1))[0]
    summ["RASE"] += torch.Tensor.tolist(tmp[:,5].reshape(1,-1))[0]
    summ["SIFT"] += torch.Tensor.tolist(tmp[:,6].reshape(1,-1))[0]
    summ["SSIM"] += torch.Tensor.tolist(tmp[:,7].reshape(1,-1))[0]
    summ["DTW"] += torch.Tensor.tolist(tmp[:,8].reshape(1,-1))[0]
    
summ = pd.DataFrame(summ)
print("Metrics of Classic GANs")
print(summ.mean())
print(summ.min())
print(summ.max())



100%|██████████| 20/20 [01:55<00:00,  5.80s/it]

Metrics of Classic GANs
UQI        0.990611
VIFP       0.008304
SCC        0.000061
SAM        0.102923
ERGAS      2.580319
RASE     220.102630
SIFT      20.731250
SSIM       0.921823
DTW       92.047467
dtype: float64
UQI        0.986582
VIFP       0.000134
SCC       -0.003126
SAM        0.090009
ERGAS      2.254309
RASE     173.048996
SIFT       4.000000
SSIM       0.898585
DTW        4.405329
dtype: float64
UQI        0.992879
VIFP       0.053611
SCC        0.002826
SAM        0.121127
ERGAS      3.052399
RASE     315.049469
SIFT      43.000000
SSIM       0.939116
DTW      349.964661
dtype: float64


In [16]:
from GAN_CustomLoss.modelv2 import Generator
from collections import defaultdict

model = Generator(200)
model.load_state_dict(torch.load("GAN_CustomLoss/saved_models/"+LABEL+"/generator.pth", weights_only=True) )

summ = defaultdict(list)

for i in tqdm(range(20)):
    synthetic = model(torch.randn((8,200,1)))
    
    tmp = pip.process(synthetic.detach().cpu(), samples[np.random.choice(samples.shape[0])].reshape(1, 1, 280).detach().cpu())
    
    summ["UQI"] += torch.Tensor.tolist(tmp[:,0].reshape(1,-1))[0]
    summ["VIFP"] += torch.Tensor.tolist(tmp[:,1].reshape(1,-1))[0]
    summ["SCC"] += torch.Tensor.tolist(tmp[:,2].reshape(1,-1))[0]
    summ["SAM"] += torch.Tensor.tolist(tmp[:,3].reshape(1,-1))[0]
    summ["ERGAS"] += torch.Tensor.tolist(tmp[:,4].reshape(1,-1))[0]
    summ["RASE"] += torch.Tensor.tolist(tmp[:,5].reshape(1,-1))[0]
    summ["SIFT"] += torch.Tensor.tolist(tmp[:,6].reshape(1,-1))[0]
    summ["SSIM"] += torch.Tensor.tolist(tmp[:,7].reshape(1,-1))[0]
    summ["DTW"] += torch.Tensor.tolist(tmp[:,8].reshape(1,-1))[0]
    
summ = pd.DataFrame(summ)
print("Metrics of GANs with Custom Loss")
print(summ.mean())
print(summ.min())
print(summ.max())



100%|██████████| 20/20 [01:55<00:00,  5.76s/it]

Metrics of GANs with Custom Loss
UQI        0.990164
VIFP       0.006532
SCC       -0.000019
SAM        0.105044
ERGAS      2.634986
RASE     231.007959
SIFT      18.775000
SSIM       0.919009
DTW       62.230681
dtype: float64
UQI        0.985858
VIFP       0.000141
SCC       -0.002824
SAM        0.088566
ERGAS      2.220233
RASE     172.804123
SIFT       3.000000
SSIM       0.891581
DTW        4.026937
dtype: float64
UQI        0.993113
VIFP       0.055389
SCC        0.002031
SAM        0.124766
ERGAS      3.144691
RASE     331.804718
SIFT      55.000000
SSIM       0.941800
DTW      269.721527
dtype: float64


In [17]:
from MD_GAN.modelv2 import Generator
from collections import defaultdict

model = Generator(200)
model.load_state_dict(torch.load("MD_GAN/saved_models/"+LABEL+"/generator.pth", weights_only=True) )

summ = defaultdict(list)

for i in tqdm(range(20)):
    synthetic = model(torch.randn((8,200,1)))
    
    tmp = pip.process(synthetic.detach().cpu(), samples[np.random.choice(samples.shape[0])].reshape(1, 1, 280).detach().cpu())
    
    summ["UQI"] += torch.Tensor.tolist(tmp[:,0].reshape(1,-1))[0]
    summ["VIFP"] += torch.Tensor.tolist(tmp[:,1].reshape(1,-1))[0]
    summ["SCC"] += torch.Tensor.tolist(tmp[:,2].reshape(1,-1))[0]
    summ["SAM"] += torch.Tensor.tolist(tmp[:,3].reshape(1,-1))[0]
    summ["ERGAS"] += torch.Tensor.tolist(tmp[:,4].reshape(1,-1))[0]
    summ["RASE"] += torch.Tensor.tolist(tmp[:,5].reshape(1,-1))[0]
    summ["SIFT"] += torch.Tensor.tolist(tmp[:,6].reshape(1,-1))[0]
    summ["SSIM"] += torch.Tensor.tolist(tmp[:,7].reshape(1,-1))[0]
    summ["DTW"] += torch.Tensor.tolist(tmp[:,8].reshape(1,-1))[0]
    
summ = pd.DataFrame(summ)
print("Metrics of MD-GAN")
print(summ.mean())
print(summ.min())
print(summ.max())


100%|██████████| 20/20 [01:57<00:00,  5.87s/it]

Metrics of MD-GAN
UQI        0.989436
VIFP       0.005860
SCC       -0.000089
SAM        0.108493
ERGAS      2.723662
RASE     248.072347
SIFT      20.818750
SSIM       0.913960
DTW      132.818187
dtype: float64
UQI        0.985401
VIFP       0.000158
SCC       -0.003118
SAM        0.092209
ERGAS      2.310654
RASE     182.604065
SIFT       3.000000
SSIM       0.885977
DTW        9.426184
dtype: float64
UQI        0.992463
VIFP       0.038680
SCC        0.001468
SAM        0.127405
ERGAS      3.217639
RASE     347.871582
SIFT      65.000000
SSIM       0.937363
DTW      375.545013
dtype: float64
